In [ ]:
import os
from pyproj import Transformer
from pathlib import Path
from rasterio.mask import mask
from cutImg import openWaypointFile, clip_raster
from preprocessCutPoint import lat_lon_to_epsg32654

def getPix4dRGBImgPath(filePath):
    RGBPathList = []
    DSMPathList = []

    for dateFileName in os.listdir(filePath):
        dateFileNamePath = filePath + '/' + dateFileName

        for step1 in os.listdir(dateFileNamePath):
            step1FilePath = dateFileNamePath + '/' + step1
            if Path(step1FilePath).is_dir():

                for step2 in os.listdir(step1FilePath):
                    if(step2 == "3_dsm_ortho"):
                        step2FilePath = step1FilePath + '/' + step2

                        for step3 in os.listdir(step2FilePath):
                            step3FilePath = step2FilePath + '/' + step3
                            if(step3 == "1_dsm"):
                                for step4 in os.listdir(step3FilePath):
                                    if(step4[-4:] == ".tif"):
                                        step4FilePath = step3FilePath + '/' + step4
                                        DSMPathList.append(step4FilePath)

                            if(step3 == "2_mosaic"):

                                for step4 in os.listdir(step3FilePath):
                                    if(step4[-4:] == ".tif"):
                                        step4FilePath = step3FilePath + '/' + step4
                                        RGBPathList.append(step4FilePath)
    return RGBPathList, DSMPathList

def openOffset(path):
    content = open(path,'r').read()
    content = content.split("\n")
    newDict = {}
    for line in content:
        line = line.split(' ')
        x,y = lat_lon_to_epsg32654(float(line[1]), float(line[2]))
        newDict[line[0]] = [float(line[1]), float(line[2])]
    return newDict
                
def writeWayPointFile(content, fileName):
    # print(content)
    file = open(fileName, "a")
    file.write("<?xml version='1.0' encoding='utf-8' ?>")
    file.write("\n")
    file.write('<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:b="http://www.bizstation.jp/waypoint_extension" creator="Drogger GPS for Android 2.12.229" version="1.1">"')

    for x in content:
        for point in x:
            transformer = Transformer.from_crs("EPSG:32654", "EPSG:4326", always_xy=True)
            lon, lat = transformer.transform(point[0], point[1])
            writeContent = '<wpt lat="' + str(lat) + '" lon="' + str(lon) + '">'
            file.write("\n")
            file.write(writeContent)
            file.write("\n")
            file.write("</wpt>")
            # <wpt lat="38.255167122" lon="140.828141869">
            # </wpt>

    file.write("\n")
    file.write("</gpx>")
    file.close()

def calculateOffsetForCoor(coordinateList, coordinateOffsetDict):
    allCoorDict = {}
    for key, offsets in sorted(coordinateOffsetDict.items()):
        newCoordinateList = []
        for eachSet in coordinateList:
            newSetList = []
            for eachPoint in eachSet:

                transformer = Transformer.from_crs("EPSG:32654", "EPSG:4326", always_xy=True)
                lon, lat = transformer.transform(eachPoint[0], eachPoint[1])
                x = lon + offsets[1]
                y = lat + offsets[0]
                transformer = Transformer.from_crs("EPSG:4326", "EPSG:32654", always_xy=True)
                x, y = transformer.transform(x, y)
                
                newSetList.append([x,y])
            newCoordinateList.append(newSetList)
        allCoorDict[key] = newCoordinateList
    return allCoorDict

def loopCalClipRaster(coordinateList, imagePath, outputFolderPath, type, key):
    outputFolderPath = outputFolderPath + '/' + type + key
    if not os.path.exists(outputFolderPath):
        os.makedirs(outputFolderPath)
    count = 1
    for smallImageCoorList in coordinateList:
        subOutputFolderPath = outputFolderPath + '/' + type + key + "_" + str(count)
        if not os.path.exists(subOutputFolderPath):
            os.makedirs(subOutputFolderPath)
        else:
            for file in os.listdir(subOutputFolderPath):
                if(file[-4:] == ".xml"):
                    os.remove(subOutputFolderPath+'/'+file)
                    print("removed ", subOutputFolderPath+'/'+file)
        output_path = subOutputFolderPath + "/" + type + key + "_" + str(count) + "_" + "original" +".tif"
        if os.path.exists(output_path):
            os.remove(output_path)
        clip_raster(imagePath, smallImageCoorList, output_path)
        # print(output_path, count, '/', '115')
        count += 1

In [ ]:
RGBFilePath = "D:/ice-wheat/data/Processed/MAVIC-RGB"
RGBFilePathList, DSMFilePathList = getPix4dRGBImgPath(RGBFilePath)
RGBFilePathList = RGBFilePathList[1:]
DSMFilePathList = DSMFilePathList[1:]
coordinateList = openWaypointFile("preprocessedCutPointFile.txt")
coordinateOffsetDict = openOffset("C:/Users/pacha/Desktop/masterProj/MasterProj/rawImage/offset.txt")
allCoorDict = calculateOffsetForCoor(coordinateList, coordinateOffsetDict)

In [ ]:
for mainCount, key in enumerate([*allCoorDict]):
    print(mainCount, ":", key)
    loopCalClipRaster(allCoorDict[key], RGBFilePathList[mainCount], "D:/ice-wheat/data/dataForProcess/mainData/RGB", "RGB_", key)
    loopCalClipRaster(allCoorDict[key], DSMFilePathList[mainCount], "D:/ice-wheat/data/dataForProcess/mainData/DSM", "DSM_", key)



In [ ]:
# loopCalClipRaster(allCoorDict['202406111255'], RGBFilePathList[-4], "D:/ice-wheat/data/dataForProcess/mainData/test", "RGB_", "202406111255")

In [ ]:
# for i, j in allCoorDict.items():
#     writeWayPointFile(j, i+'.gpx')